### 0. Imports

In [2066]:
import pandas as pd
import numpy as np
from unidecode import unidecode

### 1. Introduction to this notebook

# 2. Transformation

## 2.0 Airports

Change column names to lowercase

In [2067]:
airports = pd.read_csv("../data/airport_codes/countries_airports.csv", index_col=0)
airports.head()

,country,city,city_entityId,airport_skyId,airport_entityId,airport_name,latitude,longitude
0,spain,Madrid,27544850,MAD,95565077,Madrid,40.416705,-3.703582
1,spain,Barcelona,27548283,BCN,95565085,Barcelona,41.382894,2.177432
2,spain,Port Of Spain,27546011,POS,104120358,Port Of Spain,NaN,NaN
3,spain,Málaga,27547484,AGP,95565095,Malaga,36.721303,-4.421637
4,spain,Seville,27547022,SVQ,95565089,Seville,37.388630,-5.995340


In [2068]:
os.makedirs("../data/airport_codes/transformed/",exist_ok=True)

In [2069]:
airports.rename(columns={"city":"city_name"},inplace=True)

In [2070]:
airports.columns = [column.lower() for column in airports.columns]

In [2071]:
airports[["city_name","airport_name"]] = airports[["city_name","airport_name"]].applymap(lambda x: unidecode(x.lower()))

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\1956444939.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  airports[["city_name","airport_name"]] = airports[["city_name","airport_name"]].applymap(lambda x: unidecode(x.lower()))


In [2072]:
airports.to_parquet("../data/airport_codes/transformed/countries_airports.parquet")

## 2.1 Flights

In [2073]:
itineraries = pd.read_parquet("../data/flights/itineraries.parquet")
itineraries.info()
itineraries.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16552 entries, 0 to 16551
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   itinerary_id                  16552 non-null  object        
 1   query_date                    16552 non-null  datetime64[ns]
 2   score                         16552 non-null  float64       
 3   duration                      16552 non-null  int64         
 4   price                         16552 non-null  int64         
 5   price_currency                16552 non-null  object        
 6   stops                         16552 non-null  int64         
 7   departure                     16552 non-null  datetime64[ns]
 8   arrival                       16552 non-null  datetime64[ns]
 9   company                       16552 non-null  object        
 10  self_transfer                 16552 non-null  bool          
 11  fare_is_change_allowed      

,itinerary_id,query_date,score,duration,price,price_currency,stops,departure,arrival,company,...,fare_is_change_allowed,fare_is_partially_changeable,fare_is_cancellation_allowed,fare_is_partially_refundable,origin_airport,destination_airport,origin_airport_code,destination_airport_code,origin_airport_entityid,destination_airport_entityid
0,13870-2411081510--32680-0-9772-2411081635,2024-11-04 05:52:24.873861,0.999000,85,49,€,0,2024-11-08 15:10:00,2024-11-08 16:35:00,Air Europa,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
1,13870-2411080715--32222-0-9772-2411080830,2024-11-04 05:52:24.879866,0.864733,75,38,€,0,2024-11-08 07:15:00,2024-11-08 08:30:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
2,13870-2411081455--32222-0-9772-2411081610,2024-11-04 05:52:24.881888,0.707501,75,63,€,0,2024-11-08 14:55:00,2024-11-08 16:10:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
3,13870-2411080745--32222-0-9772-2411080900,2024-11-04 05:52:24.881888,0.553361,75,40,€,0,2024-11-08 07:45:00,2024-11-08 09:00:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
4,13870-2411080830--32222-0-9772-2411080945,2024-11-04 05:52:24.883025,0.551953,75,47,€,0,2024-11-08 08:30:00,2024-11-08 09:45:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085


Apparently, all data types and values come out correctly from the Air Scraper API. Something that is missing and would have to be provided to the user would be the suggested url to get the flights.

In [2074]:
itineraries

,itinerary_id,query_date,score,duration,price,price_currency,stops,departure,arrival,company,...,fare_is_change_allowed,fare_is_partially_changeable,fare_is_cancellation_allowed,fare_is_partially_refundable,origin_airport,destination_airport,origin_airport_code,destination_airport_code,origin_airport_entityid,destination_airport_entityid
0,13870-2411081510--32680-0-9772-2411081635,2024-11-04 05:52:24.873861,0.999000,85,49,€,0,2024-11-08 15:10:00,2024-11-08 16:35:00,Air Europa,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
1,13870-2411080715--32222-0-9772-2411080830,2024-11-04 05:52:24.879866,0.864733,75,38,€,0,2024-11-08 07:15:00,2024-11-08 08:30:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
2,13870-2411081455--32222-0-9772-2411081610,2024-11-04 05:52:24.881888,0.707501,75,63,€,0,2024-11-08 14:55:00,2024-11-08 16:10:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
3,13870-2411080745--32222-0-9772-2411080900,2024-11-04 05:52:24.881888,0.553361,75,40,€,0,2024-11-08 07:45:00,2024-11-08 09:00:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
4,13870-2411080830--32222-0-9772-2411080945,2024-11-04 05:52:24.883025,0.551953,75,47,€,0,2024-11-08 08:30:00,2024-11-08 09:45:00,Iberia,...,False,False,False,False,Madrid,Barcelona,MAD,BCN,95565077,95565085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16547,"17543-2510260600--31781,-32222-1-13870-2510261010",2024-11-04 05:52:38.196143,0.223096,250,123,€,1,2025-10-26 06:00:00,2025-10-26 10:10:00,TAP Air Portugal,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16548,17543-2510261250--32478-1-13870-2510262000,2024-11-04 05:52:38.196143,0.194155,430,388,€,1,2025-10-26 12:50:00,2025-10-26 20:00:00,Brussels Airlines,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16549,17543-2510261950--31799-1-13870-2510270915,2024-11-04 05:52:38.197142,0.137538,805,467,€,1,2025-10-26 19:50:00,2025-10-27 09:15:00,SWISS,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077
16550,17543-2510261240--31799-1-13870-2510270915,2024-11-04 05:52:38.197142,0.091166,1235,467,€,1,2025-10-26 12:40:00,2025-10-27 09:15:00,SWISS,...,False,False,False,False,Valencia,Madrid,VLC,MAD,95565090,95565077


In [2075]:
rename_columns_dict = {
    "fare_isChangeAllowed":"fare_is_change_allowed",
    "fare_isPartiallyChangeable": "fare_is_partially_changeable",
    "fare_isCancellationAllowed": "fare_is_cancellation_allowed",
    "fare_isPartiallyRefundable": "fare_is_partially_refundable"
}
itineraries.rename(columns=rename_columns_dict,inplace=True)

## 2.2 Accommodations

In [2076]:
booking_df = pd.read_parquet("../data/accommodations/booking.parquet")
booking_df.info()
booking_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22980 entries, 0 to 22979
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   query_date               22980 non-null  datetime64[ns]
 1   city                     22980 non-null  object        
 2   checkin                  22980 non-null  object        
 3   checkout                 22980 non-null  object        
 4   n_adults_search          22980 non-null  object        
 5   n_children_search        22980 non-null  object        
 6   n_rooms_search           22980 non-null  object        
 7   name                     22980 non-null  object        
 8   url                      22980 non-null  object        
 9   price_currency           22980 non-null  object        
 10  total_price_amount       22980 non-null  object        
 11  distance_city_center_km  22980 non-null  object        
 12  score                    22306 n

,query_date,city,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,name,url,price_currency,...,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi
0,2024-11-04 00:43:02.045730,barcelona,2024-11-08,2024-11-10,2,0,1,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,€,...,True,False,Habitación Doble Económica,True,False,False,False,False,None,False
1,2024-11-04 00:43:02.239603,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,€,...,True,False,Habitación (1 o 2 adultos) - 1 o 2 camas,True,True,False,False,False,None,False
2,2024-11-04 00:43:02.245775,barcelona,2024-11-08,2024-11-10,2,0,1,Travelodge Barcelona Poblenou,https://www.booking.com/hotel/es/travelodge-ba...,€,...,True,False,Habitación Doble,True,False,False,False,False,None,False
3,2024-11-04 00:43:02.266016,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Best Front Maritim,https://www.booking.com/hotel/es/front-maritim...,€,...,True,True,Habitación Doble - 2 camas,False,True,False,False,False,None,False
4,2024-11-04 00:43:02.269974,barcelona,2024-11-08,2024-11-10,2,0,1,Apartamentos DV,https://www.booking.com/hotel/es/apartamentos-...,€,...,True,False,Habitación Doble con baño privado,True,False,False,False,False,8.9,False


Convert checkin and checkout to datetime

In [2077]:
booking_df[["checkin","checkout"]] = booking_df[["checkin","checkout"]].astype("datetime64[ns]")

In [2078]:
booking_df["room_type"].value_counts()[:20]

room_type
Apartamento de 1 dormitorio                        4383
Habitación Doble                                   1925
Apartamento de 2 dormitorios                       1694
Apartamento                                        1621
Habitación Doble - 1 o 2 camas                     1520
Habitación Doble con baño privado                   611
Habitación Doble Económica                          545
Habitación Doble con baño compartido                523
Habitación Doble - 2 camas                          512
Apartamento de 3 dormitorios                        355
Estudio                                             324
Habitación Doble Estándar - 1 o 2 camas             316
Habitación Doble Estándar                           309
Habitación Doble Pequeña                            240
Apartamento Deluxe                                  232
Habitación Doble con baño compartido - 2 camas      229
Habitación Deluxe                                   184
Habitación Doble Superior             

Standardize room types

In [2079]:
booking_df

,query_date,city,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,name,url,price_currency,...,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi
0,2024-11-04 00:43:02.045730,barcelona,2024-11-08,2024-11-10,2,0,1,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,€,...,True,False,Habitación Doble Económica,True,False,False,False,False,None,False
1,2024-11-04 00:43:02.239603,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,€,...,True,False,Habitación (1 o 2 adultos) - 1 o 2 camas,True,True,False,False,False,None,False
2,2024-11-04 00:43:02.245775,barcelona,2024-11-08,2024-11-10,2,0,1,Travelodge Barcelona Poblenou,https://www.booking.com/hotel/es/travelodge-ba...,€,...,True,False,Habitación Doble,True,False,False,False,False,None,False
3,2024-11-04 00:43:02.266016,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Best Front Maritim,https://www.booking.com/hotel/es/front-maritim...,€,...,True,True,Habitación Doble - 2 camas,False,True,False,False,False,None,False
4,2024-11-04 00:43:02.269974,barcelona,2024-11-08,2024-11-10,2,0,1,Apartamentos DV,https://www.booking.com/hotel/es/apartamentos-...,€,...,True,False,Habitación Doble con baño privado,True,False,False,False,False,8.9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22975,2024-11-04 00:46:18.554270,valencia,2025-10-31,2025-11-02,2,0,1,La Tía Homey Haven - Host Family,https://www.booking.com/hotel/es/latia-homeyha...,€,...,True,False,Habitación Doble,True,False,False,False,False,None,False
22976,2024-11-04 00:46:18.558172,valencia,2025-10-31,2025-11-02,2,0,1,NAP v,https://www.booking.com/hotel/es/goa-valencia....,€,...,True,False,Cama individual en dormitorio mixto,False,True,False,False,False,None,False
22977,2024-11-04 00:46:18.561873,valencia,2025-10-31,2025-11-02,2,0,1,River Hostel,https://www.booking.com/hotel/es/the-river-hos...,€,...,True,False,Litera en habitación compartida de 12 camas,False,True,False,False,False,None,False
22978,2024-11-04 00:46:18.564989,valencia,2025-10-31,2025-11-02,2,0,1,The POD Suite Hostel Blasco,https://www.booking.com/hotel/es/the-pod-suite...,€,...,True,False,A01 Female Lower bed In shared room with 6 beds,False,True,False,False,False,9.3,False


In [2080]:
conditions = [
    booking_df["room_type"].str.startswith("Apartamento").values.astype(bool),
    booking_df["room_type"].str.startswith("Cama en habitacion compartida").values.astype(bool),
    booking_df["room_type"].str.startswith("Habitacion Doble Superior").values.astype(bool),
    booking_df["room_type"].str.startswith("Habitacion Doble").values.astype(bool)
]

choices = [
    "Apartamento",
    "Cama en habitacion compartida",  
    "Habitacion Doble Superior",
    "Habitacion Doble"
]


booking_df["standardized_room_type"] = np.select(conditions, choices, default=booking_df["room_type"])


In [2081]:
booking_df

,query_date,city,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,name,url,price_currency,...,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi,standardized_room_type
0,2024-11-04 00:43:02.045730,barcelona,2024-11-08,2024-11-10,2,0,1,Nice and comfortable room for your stay in BCN,https://www.booking.com/hotel/es/nice-and-comf...,€,...,False,Habitación Doble Económica,True,False,False,False,False,None,False,Habitación Doble Económica
1,2024-11-04 00:43:02.239603,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Derby,https://www.booking.com/hotel/es/derby.es.html...,€,...,False,Habitación (1 o 2 adultos) - 1 o 2 camas,True,True,False,False,False,None,False,Habitación (1 o 2 adultos) - 1 o 2 camas
2,2024-11-04 00:43:02.245775,barcelona,2024-11-08,2024-11-10,2,0,1,Travelodge Barcelona Poblenou,https://www.booking.com/hotel/es/travelodge-ba...,€,...,False,Habitación Doble,True,False,False,False,False,None,False,Habitación Doble
3,2024-11-04 00:43:02.266016,barcelona,2024-11-08,2024-11-10,2,0,1,Hotel Best Front Maritim,https://www.booking.com/hotel/es/front-maritim...,€,...,True,Habitación Doble - 2 camas,False,True,False,False,False,None,False,Habitación Doble - 2 camas
4,2024-11-04 00:43:02.269974,barcelona,2024-11-08,2024-11-10,2,0,1,Apartamentos DV,https://www.booking.com/hotel/es/apartamentos-...,€,...,False,Habitación Doble con baño privado,True,False,False,False,False,8.9,False,Habitación Doble con baño privado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22975,2024-11-04 00:46:18.554270,valencia,2025-10-31,2025-11-02,2,0,1,La Tía Homey Haven - Host Family,https://www.booking.com/hotel/es/latia-homeyha...,€,...,False,Habitación Doble,True,False,False,False,False,None,False,Habitación Doble
22976,2024-11-04 00:46:18.558172,valencia,2025-10-31,2025-11-02,2,0,1,NAP v,https://www.booking.com/hotel/es/goa-valencia....,€,...,False,Cama individual en dormitorio mixto,False,True,False,False,False,None,False,Cama individual en dormitorio mixto
22977,2024-11-04 00:46:18.561873,valencia,2025-10-31,2025-11-02,2,0,1,River Hostel,https://www.booking.com/hotel/es/the-river-hos...,€,...,False,Litera en habitación compartida de 12 camas,False,True,False,False,False,None,False,Litera en habitación compartida de 12 camas
22978,2024-11-04 00:46:18.564989,valencia,2025-10-31,2025-11-02,2,0,1,The POD Suite Hostel Blasco,https://www.booking.com/hotel/es/the-pod-suite...,€,...,False,A01 Female Lower bed In shared room with 6 beds,False,True,False,False,False,9.3,False,A01 Female Lower bed In shared room with 6 beds


Apartamento
Cama en habitacion compartida
Habitacion Doble Superior
Habitacion Doble 

In [2082]:
booking_df["shared_bathroom"] = np.where(booking_df["room_type"].str.contains("baño compartido"),True,False)

Add separate info about balcony

In [2083]:
booking_df["balcony"] = np.where(booking_df["room_type"].str.contains("balcón"),True,False)

In [2084]:
booking_df["sustainability_cert"].value_counts()

sustainability_cert
False    21468
True      1512
Name: count, dtype: int64

In [2085]:
booking_df["free_cancellation"].value_counts()

free_cancellation
False    19162
True      3818
Name: count, dtype: int64

In [2086]:
booking_df["breakfast_included"].value_counts()

breakfast_included
False    19162
True      3818
Name: count, dtype: int64

Total price per night

In [2087]:
booking_df["price_night"] = booking_df["total_price_amount"].astype(int) / (booking_df["checkout"] - booking_df["checkin"]).dt.days

In [2088]:
booking_df["price_night"]

0         74.0
1        136.0
2        128.5
3        132.5
4        146.0
         ...  
22975     72.0
22976     70.0
22977     49.5
22978     84.0
22979     32.5
Name: price_night, Length: 22980, dtype: float64

In [2089]:
booking_df["distance_city_center_km"] = booking_df["distance_city_center_km"].astype(float)

In [2090]:
booking_df["distance_city_center_km"] = np.where(booking_df["distance_city_center_km"] > 10, booking_df["distance_city_center_km"]/1000, booking_df["distance_city_center_km"])

In [2091]:
booking_df[["close_to_metro","sustainability_cert"]] = booking_df[["close_to_metro","sustainability_cert"]].astype(bool)

In [2092]:
booking_df["n_comments"] = booking_df["n_comments"].fillna(0).astype(int)

## 2.3 Activities

In [2093]:
activities_df = pd.read_parquet("../data/activities/activities.parquet")

In [2094]:
activities_df

,query_date,city,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-04 01:23:44.031007,barcelona,2024-11-05,2024-11-11,Excursión a Montserrat + Visita a una bodega,En esta excursión a Montserrat no solo disfrut...,www.civitatis.com/es/barcelona/tour-tapas-vino...,www.civitatis.com/f/espana/barcelona/tour-tapa...,None,"[06, 11, 07, 09, 08, 05, 10]","[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",7h 30m,41.3940236912484,2.181866082214644,19.98,EUR,Gastronomía y enoturismo,Español
1,2024-11-04 01:23:44.034541,barcelona,2024-11-05,2024-11-11,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/paseo-cat...,"[07, 08, 06, 11, 10, 09, 05]","[[16:30], [16:30], [17:00], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
2,2024-11-04 01:23:44.037536,barcelona,2024-11-05,2024-11-11,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/free-tour...,"[11, 05, 06, 07, 08]","[[11:30], [11:30], [11:30], [11:30], [11:30]]",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español
3,2024-11-04 01:23:44.039537,barcelona,2024-11-05,2024-11-11,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/billete-t...,"[07, 11, 05, 08, 06, 09, 10]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-04 01:23:44.043574,barcelona,2024-11-05,2024-11-11,Excursión a Tarragona y Sitges,En esta excursión a Tarragona y Sitges desde B...,www.civitatis.com/es/barcelona/excursion-tarra...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/excursion...,"[05, 07, 10, 09, 08, 06]","[[8:30], [8:30], [8:30], [8:30], [8:30], [8:30]]",10 horas,0,0,24.63,EUR,Excursiones de un día,Español
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446,2024-11-04 01:24:04.173259,valencia,2025-10-25,2025-10-31,Paseo en catamarán por Valencia,En este paseo en catamarán a vela por Valencia...,www.civitatis.com/es/valencia/paseo-catamaran-...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/valencia/paseo-cata...,None,None,1 hora,39.460925,-0.324599,3.00,EUR,Paseos en barco,None
18447,2024-11-04 01:24:04.174238,valencia,2025-10-25,2025-10-31,Entrada al Museo Iluziona de Valencia,Descubrid las leyendas de Valencia a través de...,www.civitatis.com/es/valencia/entrada-museo-il...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/valencia/entrada-mu...,None,None,None,39.46475896364761,-0.3758850218933492,3.00,EUR,Entradas,None
18448,2024-11-04 01:24:04.175239,valencia,2025-10-25,2025-10-31,Excursión a Montanejos,¿Os gusta caminar por la naturaleza? En esta e...,www.civitatis.com/es/valencia/excursion-montan...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/valencia/excursion-...,None,None,8h 30m,39.47625373037879,-0.3469842302587606,11.25,EUR,Excursiones de un día,Español
18449,2024-11-04 01:24:04.176238,valencia,2025-10-25,2025-10-31,Tour del estadio de Mestalla,Acompañadnos a descubrir los secretos de uno d...,www.civitatis.com/es/valencia/tour-estadio-mes...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/valencia/tour-estad...,None,None,1 hora,39.474775,-0.359234,2.60,EUR,Visitas guiadas y free tours,Español


### 2.3.1 Cleaning neccessary

1. Remove duplicates
2. Drop activities without name
3. Keep only the good image: The second if it has 2, otherwise the first.
4. Take unique, non null and non zero latitudes and longitudes, to convert to address.
5. Format the available dates and times

Careful as some non-available times might mean they do not apply, i.e. tickets.

##### 2.3.1.1 Remove duplicates

By activity_name and activity_date_range_start

In [2095]:
activities_df[activities_df.duplicated(subset=["activity_date_range_start","activity_name"],keep=False)].sort_values(by="activity_name").head()

,query_date,city,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
14191,2024-11-04 01:23:59.839602,sevilla,2025-09-19,2025-09-25,Alquiler de bicicletas en Sevilla,Recorrer Sevilla en bicicleta es una experienc...,www.civitatis.com/es/sevilla/alquiler-biciclet...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/sevilla/alquiler-bi...,"[20, 22, 24, 25, 19, 23, 21]","[[], [], [], [], [], [], []]",3h -7d,0,0,2.0,EUR,Transporte y traslados,None
14293,2024-11-04 01:23:59.916506,sevilla,2025-09-25,2025-10-01,Alquiler de bicicletas en Sevilla,Recorrer Sevilla en bicicleta es una experienc...,www.civitatis.com/es/sevilla/alquiler-biciclet...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/sevilla/alquiler-bi...,"[30, 27, 26, 28, 29, 25, 01]","[[], [], [], [], [], [], []]",3h -7d,0,0,2.0,EUR,Transporte y traslados,None
13968,2024-11-04 01:23:59.625676,sevilla,2025-09-07,2025-09-13,Alquiler de bicicletas en Sevilla,Recorrer Sevilla en bicicleta es una experienc...,www.civitatis.com/es/sevilla/alquiler-biciclet...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/sevilla/alquiler-bi...,"[12, 11, 08, 13, 09, 10, 07]","[[], [], [], [], [], [], []]",3h -7d,0,0,2.0,EUR,Transporte y traslados,None
14070,2024-11-04 01:23:59.734632,sevilla,2025-09-13,2025-09-19,Alquiler de bicicletas en Sevilla,Recorrer Sevilla en bicicleta es una experienc...,www.civitatis.com/es/sevilla/alquiler-biciclet...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/sevilla/alquiler-bi...,"[19, 15, 18, 17, 13, 14, 16]","[[], [], [], [], [], [], []]",3h -7d,0,0,2.0,EUR,Transporte y traslados,None
13866,2024-11-04 01:23:59.497955,sevilla,2025-09-01,2025-09-07,Alquiler de bicicletas en Sevilla,Recorrer Sevilla en bicicleta es una experienc...,www.civitatis.com/es/sevilla/alquiler-biciclet...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/sevilla/alquiler-bi...,"[04, 02, 05, 03, 06, 01, 07]","[[], [], [], [], [], [], []]",3h -7d,0,0,2.0,EUR,Transporte y traslados,None


In [2096]:
activities_df.drop_duplicates(subset=["activity_date_range_start","activity_name"], inplace=True)

#### 2.3.1.2 Drop activities without name

In [2097]:
activities_df = activities_df[~activities_df["activity_name"].isna()]

In [2098]:
activities_df["activity_name"].isna().sum()

np.int64(0)

#### 2.3.1.2 Keep only the good image: The second if it has 2, otherwise the first.

In [2099]:
activities_df["image"] = np.where(~activities_df["image2"].isna(), activities_df["image2"], activities_df["image"])
activities_df.drop(columns="image2",inplace=True)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\2977301122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df["image"] = np.where(~activities_df["image2"].isna(), activities_df["image2"], activities_df["image"])
C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\2977301122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df.drop(columns="image2",inplace=True)


In [2100]:
activities_df.head()

,query_date,city,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-04 01:23:44.031007,barcelona,2024-11-05,2024-11-11,Excursión a Montserrat + Visita a una bodega,En esta excursión a Montserrat no solo disfrut...,www.civitatis.com/es/barcelona/tour-tapas-vino...,www.civitatis.com/f/espana/barcelona/tour-tapa...,"[06, 11, 07, 09, 08, 05, 10]","[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",7h 30m,41.3940236912484,2.181866082214644,19.98,EUR,Gastronomía y enoturismo,Español
1,2024-11-04 01:23:44.034541,barcelona,2024-11-05,2024-11-11,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[07, 08, 06, 11, 10, 09, 05]","[[16:30], [16:30], [17:00], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
2,2024-11-04 01:23:44.037536,barcelona,2024-11-05,2024-11-11,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[11, 05, 06, 07, 08]","[[11:30], [11:30], [11:30], [11:30], [11:30]]",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español
3,2024-11-04 01:23:44.039537,barcelona,2024-11-05,2024-11-11,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[07, 11, 05, 08, 06, 09, 10]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-04 01:23:44.043574,barcelona,2024-11-05,2024-11-11,Excursión a Tarragona y Sitges,En esta excursión a Tarragona y Sitges desde B...,www.civitatis.com/es/barcelona/excursion-tarra...,www.civitatis.com/f/espana/barcelona/excursion...,"[05, 07, 10, 09, 08, 06]","[[8:30], [8:30], [8:30], [8:30], [8:30], [8:30]]",10 horas,0,0,24.63,EUR,Excursiones de un día,Español


#### 2.3.1.3 Take unique, non null and non zero latitudes and longitudes, to convert to address.

In [2101]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [2102]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2103]:
import aiohttp
import asyncio

async def get_address(lat: float, lon: float, api_key: str) -> str:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
            
            if data.get("status") == "OK":
                address = data["results"][0]["formatted_address"]
                return address
            else:
                return "Address not found"

# Example usage
async def get_adresses_async(lat_lon_df):
    api_key = GOOGLE_API_KEY
    lat_lon_tasks = [get_address(lat, lon, api_key) for lat, lon in lat_lon_df.itertuples(index=False, name=None)]
    addresses = await asyncio.gather(*lat_lon_tasks)
    return addresses


In [2104]:
not_nan = activities_df[["latitude", "longitude"]].notna().all(axis=1)
unique_combinations = activities_df.loc[not_nan,["latitude","longitude"]].drop_duplicates()

In [2105]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [2106]:
unique_combinations["lat_lon"] = list(zip(unique_combinations.latitude, unique_combinations.longitude))
activities_df["lat_lon"] = list(zip(activities_df.latitude, activities_df.longitude))

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\3996858165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df["lat_lon"] = list(zip(activities_df.latitude, activities_df.longitude))


In [2107]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [2108]:
lat_lon_to_address_ampping = dict(zip(unique_combinations["lat_lon"], unique_combinations["address"]))

In [2109]:
activities_df["address"] = activities_df["lat_lon"].map(lat_lon_to_address_ampping)
activities_df.drop(columns="lat_lon",inplace=True)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\388804089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df["address"] = activities_df["lat_lon"].map(lat_lon_to_address_ampping)
C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\388804089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df.drop(columns="lat_lon",inplace=True)


In [2110]:
activities_df.head(5)

,query_date,city,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish,address
0,2024-11-04 01:23:44.031007,barcelona,2024-11-05,2024-11-11,Excursión a Montserrat + Visita a una bodega,En esta excursión a Montserrat no solo disfrut...,www.civitatis.com/es/barcelona/tour-tapas-vino...,www.civitatis.com/f/espana/barcelona/tour-tapa...,"[06, 11, 07, 09, 08, 05, 10]","[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",7h 30m,41.3940236912484,2.181866082214644,19.98,EUR,Gastronomía y enoturismo,Español,"Barcelona (Bus Terminal Nord), Eixample, 08013..."
1,2024-11-04 01:23:44.034541,barcelona,2024-11-05,2024-11-11,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[07, 08, 06, 11, 10, 09, 05]","[[16:30], [16:30], [17:00], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español,"Moll de les Drassanes, 3P, Ciutat Vella, 08039..."
2,2024-11-04 01:23:44.037536,barcelona,2024-11-05,2024-11-11,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[11, 05, 06, 07, 08]","[[11:30], [11:30], [11:30], [11:30], [11:30]]",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español,"Ctra. del Carmel, 23, Horta-Guinardó, 08024 Ba..."
3,2024-11-04 01:23:44.039537,barcelona,2024-11-05,2024-11-11,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[07, 11, 05, 08, 06, 09, 10]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None,"Avinguda Miramar, 30, Sants-Montjuïc, 08038 Ba..."
4,2024-11-04 01:23:44.043574,barcelona,2024-11-05,2024-11-11,Excursión a Tarragona y Sitges,En esta excursión a Tarragona y Sitges desde B...,www.civitatis.com/es/barcelona/excursion-tarra...,www.civitatis.com/f/espana/barcelona/excursion...,"[05, 07, 10, 09, 08, 06]","[[8:30], [8:30], [8:30], [8:30], [8:30], [8:30]]",10 horas,0,0,24.63,EUR,Excursiones de un día,Español,Address not found


#### 2.3.1.4 Format the available dates and times

Available dates and times for each activity will have to go to another table to ensure database normalisation. To do that, it will be necessary to explode the df twice. 

Then, to complete the date of availability, some rules for transformation have to be established:

In [2111]:
availability_activities = activities_df[["available_days","query_date","available_times","city","activity_name","activity_date_range_start","activity_date_range_end"]]
availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20380\3535378416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)


In [2112]:
availability_activities = availability_activities.explode("paired", ignore_index=True) #.drop(columns=["available_days","available_times"])

In [2113]:
availability_activities.head()

,available_days,query_date,available_times,city,activity_name,activity_date_range_start,activity_date_range_end,paired
0,"[06, 11, 07, 09, 08, 05, 10]",2024-11-04 01:23:44.031007,"[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,"(06, [8:45])"
1,"[06, 11, 07, 09, 08, 05, 10]",2024-11-04 01:23:44.031007,"[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,"(11, [8:45])"
2,"[06, 11, 07, 09, 08, 05, 10]",2024-11-04 01:23:44.031007,"[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,"(07, [8:45])"
3,"[06, 11, 07, 09, 08, 05, 10]",2024-11-04 01:23:44.031007,"[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,"(09, [8:45])"
4,"[06, 11, 07, 09, 08, 05, 10]",2024-11-04 01:23:44.031007,"[[8:45], [8:45], [8:45], [8:45], [8:45], [8:45...",barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11,"(08, [8:45])"


Now, there is a table with the 

In [2114]:
availability_activities[["available_days", "available_times"]] = pd.DataFrame(availability_activities["paired"].tolist(), index=availability_activities.index)
availability_activities = availability_activities.drop(columns="paired")
availability_activities

,available_days,query_date,available_times,city,activity_name,activity_date_range_start,activity_date_range_end
0,06,2024-11-04 01:23:44.031007,[8:45],barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
1,11,2024-11-04 01:23:44.031007,[8:45],barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
2,07,2024-11-04 01:23:44.031007,[8:45],barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
3,09,2024-11-04 01:23:44.031007,[8:45],barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
4,08,2024-11-04 01:23:44.031007,[8:45],barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
...,...,...,...,...,...,...,...
56169,-,2024-11-04 01:24:04.173259,-,valencia,Paseo en catamarán por Valencia,2025-10-25,2025-10-31
56170,-,2024-11-04 01:24:04.174238,-,valencia,Entrada al Museo Iluziona de Valencia,2025-10-25,2025-10-31
56171,-,2024-11-04 01:24:04.175239,-,valencia,Excursión a Montanejos,2025-10-25,2025-10-31
56172,-,2024-11-04 01:24:04.176238,-,valencia,Tour del estadio de Mestalla,2025-10-25,2025-10-31


In [2115]:
availability_activities = availability_activities.explode("available_times")
availability_activities

,available_days,query_date,available_times,city,activity_name,activity_date_range_start,activity_date_range_end
0,06,2024-11-04 01:23:44.031007,8:45,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
1,11,2024-11-04 01:23:44.031007,8:45,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
2,07,2024-11-04 01:23:44.031007,8:45,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
3,09,2024-11-04 01:23:44.031007,8:45,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
4,08,2024-11-04 01:23:44.031007,8:45,barcelona,Excursión a Montserrat + Visita a una bodega,2024-11-05,2024-11-11
...,...,...,...,...,...,...,...
56169,-,2024-11-04 01:24:04.173259,-,valencia,Paseo en catamarán por Valencia,2025-10-25,2025-10-31
56170,-,2024-11-04 01:24:04.174238,-,valencia,Entrada al Museo Iluziona de Valencia,2025-10-25,2025-10-31
56171,-,2024-11-04 01:24:04.175239,-,valencia,Excursión a Montanejos,2025-10-25,2025-10-31
56172,-,2024-11-04 01:24:04.176238,-,valencia,Tour del estadio de Mestalla,2025-10-25,2025-10-31


In [2116]:
import datetime

In [2117]:
availability_activities["activity_date_range_start"] = pd.to_datetime(availability_activities["activity_date_range_start"])
availability_activities["activity_date_range_end"] = pd.to_datetime(availability_activities["activity_date_range_end"])

availability_activities[["available_days","available_times"]] = availability_activities[["available_days","available_times"]].replace("-",None)

availability_activities["available_times"] = availability_activities["available_times"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M").time() if pd.notna(x) else None)


def calculate_adjusted_date(row):
    # return Not a Time if nan
    if pd.isna(row["available_days"]):
        return pd.NaT
    
    # convert from string
    day = int(row["available_days"]) 

    # if available date is bigger than end range, belongs to first month. Else second.
    if day > row["activity_date_range_end"].day:
        return row["activity_date_range_start"].replace(day=day)
    else:
        return row["activity_date_range_end"].replace(day=day)


availability_activities["available_date"] = availability_activities.apply(calculate_adjusted_date, axis=1)

availability_activities.drop(columns="available_days",inplace=True)



In [2118]:
availability_activities = availability_activities[~availability_activities["available_date"].isna()]

In [2119]:
availability_activities.dtypes

query_date                   datetime64[ns]
available_times                      object
city                                 object
activity_name                        object
activity_date_range_start    datetime64[ns]
activity_date_range_end      datetime64[ns]
available_date               datetime64[ns]
dtype: object

### Weather

In [2120]:
weather = pd.read_parquet("../data/weather/weather.parquet")
weather.head()

,time,apparent_temperature_mean,apparent_temperature_min,apparent_temperature_max,precipitation_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,sunshine_duration,daylight_duration,city,forecast/history
0,2024-11-03,13.3,10.8,17.2,0.0,0.0,9.7,28.4,33879.55,37286.88,Madrid,forecast
1,2024-11-04,13.6,10.4,17.9,0.1,1.0,6.5,20.2,25605.18,37149.31,Madrid,forecast
2,2024-11-05,14.6,10.7,18.8,0.3,2.0,3.5,11.2,30917.02,37013.77,Madrid,forecast
3,2024-11-06,14.2,9.7,19.5,0.0,0.0,7.6,23.4,33443.99,36880.41,Madrid,forecast
4,2024-11-07,14.0,11.5,17.5,0.0,0.0,4.0,13.7,33303.26,36749.41,Madrid,forecast


In [2121]:
weather["city"] = weather["city"].apply(lambda x: unidecode(x.lower()))
weather.head()

,time,apparent_temperature_mean,apparent_temperature_min,apparent_temperature_max,precipitation_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,sunshine_duration,daylight_duration,city,forecast/history
0,2024-11-03,13.3,10.8,17.2,0.0,0.0,9.7,28.4,33879.55,37286.88,madrid,forecast
1,2024-11-04,13.6,10.4,17.9,0.1,1.0,6.5,20.2,25605.18,37149.31,madrid,forecast
2,2024-11-05,14.6,10.7,18.8,0.3,2.0,3.5,11.2,30917.02,37013.77,madrid,forecast
3,2024-11-06,14.2,9.7,19.5,0.0,0.0,7.6,23.4,33443.99,36880.41,madrid,forecast
4,2024-11-07,14.0,11.5,17.5,0.0,0.0,4.0,13.7,33303.26,36749.41,madrid,forecast


In [2122]:
os.makedirs("../data/weather/transformed/",exist_ok=True)

In [2123]:
weather.to_parquet("../data/weather/transformed/weather.parquet")

# 3. Save for data load

## 3.1 Itineraries

In [2124]:
import os

In [2125]:
os.makedirs("../data/flights/transformed/",exist_ok=True)

In [2126]:
itineraries.to_parquet("../data/flights/transformed/itineraries.parquet")

## 3.2 Accommodations

### 3.2.1 Accommodations - booking

In [2127]:
os.makedirs("../data/accommodations/transformed/",exist_ok=True)

In [2128]:
booking_df.to_parquet("../data/accommodations/transformed/booking.parquet")

## 3.3 Activities

Activities has 2 tables, the activities per se and their available times and dates.

In [2129]:
os.makedirs("../data/activities/transformed/",exist_ok=True)

In [2130]:
activities_df.to_parquet("../data/activities/transformed/activities.parquet")
availability_activities.to_parquet("../data/activities/transformed/availabilities.parquet")